<a href="https://colab.research.google.com/github/Amit-Padye/Pytorch-/blob/main/Regression_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import torch

In [1]:
import torch.nn as nn

In [5]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [7]:
from torch.utils.data import TensorDataset

In [8]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data

In [9]:
from torch.utils.data import DataLoader

In [10]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [12]:
train_dl

In [11]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  96.,  71.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 73.,  66.,  44.],
        [ 73.,  67.,  43.]])
tensor([[104., 118.],
        [119., 133.],
        [ 22.,  37.],
        [ 57.,  69.],
        [ 56.,  70.]])


In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

###nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [13]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5402, -0.5157, -0.2149],
        [-0.1047,  0.2357,  0.3619]], requires_grad=True)
Parameter containing:
tensor([-0.0635,  0.4291], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [14]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.5402, -0.5157, -0.2149],
         [-0.1047,  0.2357,  0.3619]], requires_grad=True),
 Parameter containing:
 tensor([-0.0635,  0.4291], requires_grad=True)]

In [15]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -83.2909,   24.1350],
        [-108.3566,   34.7989],
        [-128.6279,   43.8866],
        [ -85.2922,   13.2701],
        [-101.8859,   41.1601],
        [ -83.3154,   23.7946],
        [-108.0558,   34.9252],
        [-129.3830,   44.1437],
        [ -85.2677,   13.6105],
        [-101.5605,   41.6267],
        [ -82.9900,   24.2612],
        [-108.3811,   34.4585],
        [-128.9288,   43.7603],
        [ -85.6176,   12.8034],
        [-101.8614,   41.5005]], grad_fn=<AddmmBackward>)

In [16]:
# Import nn.functional
import torch.nn.functional as F

In [17]:
# Define loss function
loss_fn = F.mse_loss

In [18]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(19079.1211, grad_fn=<MseLossBackward>)


##Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [19]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

##Train the model
We are now ready to train the model. We'll follow the same process to implement gradient descent:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [20]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [21]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 129.4572
Epoch [20/100], Loss: 170.6043
Epoch [30/100], Loss: 86.8950
Epoch [40/100], Loss: 82.4195
Epoch [50/100], Loss: 44.5005
Epoch [60/100], Loss: 41.5165
Epoch [70/100], Loss: 27.6327
Epoch [80/100], Loss: 2.3894
Epoch [90/100], Loss: 10.0717
Epoch [100/100], Loss: 18.8126
